### Assignment : Week 2
## Finding best policies in simple MDPs

Great work making the MDPs in Week 1!

In this assignment, we'll use the simplest RL techniques - Policy and Value iteration to find the best policies (which maximize the discounted total reward) in our MDPs from last week.

Feel free to use your own MDPs, or import them from the OpenAI Gym library.

You can start this assignment during/after reading Grokking Ch-3.

For this you have to install gymnasium, which is an API standard for reinforcement learning with a diverse collection of reference environments. This can be easily done by running:

    pip install gymnasium

## Frozen Lake

Let's now try to solve the Frozen Lake environment for some cases

In [142]:
# Step 0 is to import stuff

import gymnasium as gym
import numpy as np
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [143]:
# Step 1 is to get the MDP
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)
env = env.unwrapped
mdp_transitions = env.P
init_state = env.reset()
goal_state = 15

In [144]:
# Step 2 is to write the policy

# This is according to the convention of gymnasium
LEFT, DOWN, RIGHT, UP = range(4)
pi = {
    0:RIGHT, 1:RIGHT, 2:DOWN, 3:LEFT,
    4:DOWN, 5:LEFT, 6:DOWN, 7:LEFT,
    8:RIGHT, 9:RIGHT, 10:DOWN, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:RIGHT, 15:LEFT
}

# Or you can do it randomly
# pi = dict()
# for state in mdp:
#     pi[state] = np.random.choice(mdp[state].keys())

In [145]:
# Step 3 is computing the value function for this envi and policy

# Let us start with a random value function

val = dict()
for state in mdp_transitions:
    val[state] = np.random.random()

# Since 5, 7, 11, 12 and 15 are terminal states, we know their values are 0

val[5] = 0
val[7] = 0
val[11] = 0
val[12] = 0
val[15] = 0

#Or you could do it randomly, remember to set the terminal states to 0. You can also implement this while evaluating the value function using 
# val = dict()
# for state in mdp:
#     val[state] = np.random.random()
#     if mdp[state][0][0][0] == 0: # if the first action in the first outcome of the first state is 0, then it is a terminal state
#         val[state] = 0

#instead of doing thsi you can simply intialize the value function to 0 for all states 
# for state in swf_mdp:
#   val[state] = 0

In [146]:
def get_new_value_fn(val, mdp, pi, gamma = 1.0):
    
    new_val = dict()
    # Complete this function to get the new value function given the old value function and the policy

    for state in mdp:
        action = pi[state] 
        new_value = 0
        
        for prob, next_state, reward, done in mdp[state][action]:
            new_value += prob * (reward + gamma * val[next_state] * (not done))
        
        new_val[state] = new_value

    return new_val

In [147]:
#Use to above function to get the new value function, also print how many iterations it took to converge
def policy_evaluation(val, mdp, pi, epsilon=1e-10, gamma=1.0):
    count = 0
    # Complete this function to iteratively caluculate the value function until the difference between the new and old value function is less than epsilon
    # Also return the number of iterations it took to converge

    while True:
        new_val = get_new_value_fn(val, mdp, pi, gamma)
    
        delta = max(abs(new_val[state] - val[state]) for state in val)
        val = new_val
        
        count += 1
        
        if delta < epsilon:  
            break

    return val, count 

In [148]:
# Perform policy improvement using the polivy and the value function and return a new policy, the action value function should be a nested dictionary
def policy_improvement(val, mdp, pi, gamma=1.0):
    new_pi = dict()
    # q = dict()
    # Complete this function to get the new policy given the value function and the mdp

    q = {state: {} for state in mdp}  # Initialize Q(s, a) for all states and actions
    
    for state in mdp:
        for action in mdp[state]:
            q_value = 0
            for prob, next_state, reward, done in mdp[state][action]:
                q_value += prob * (reward + gamma * val[next_state] * (not done))
            q[state][action] = q_value 
        
        new_pi[state] = max(q[state], key=q[state].get)

    return new_pi, q


In [149]:
# Use the above functions to get the optimal policy and optimal value function and return the total number of iterations it took to converge
# Create a random policy and value function to start with or use the ones defined above
def policy_iteration(mdp, epsilon=1e-10, gamma=1.0):
    vval = val
    ppi = pi
    count = 0
    # Complete this function to get the optimal policy and value function and return the total number of iterations it took to converge

    while True:

        count += 1

        # Step 2: Store the old policy
        old_pi = {s: ppi[s] for s in range(len(mdp))}

        # Step 3: Policy Evaluation
        vval, _ = policy_evaluation(vval, mdp, old_pi)

        # Step 4: Policy Improvement
        ppi, __ = policy_improvement(vval, mdp, old_pi)

        # Step 5: Convergence check
        if old_pi == {s: ppi[s] for s in range(len(mdp))}:
            break

    return ppi, vval, count

In [150]:
#Now perform value iteration, note that the value function is a dictionary and not a list, also return the number of iterations it took to converge
def value_iteration(mdp, gamma=1.0, epsilon=1e-10):
    # Complete this function to get the optimal policy, optimal value function and return the total number of iterations it took to converge

    V = np.zeros(len(mdp), dtype=np.float64)
    count = 0
    
    while True:
        count += 1
        Q = np.zeros((len(mdp), len(mdp[0])), dtype=np.float64)
        
        for s in range(len(mdp)):
            for a in range(len(mdp[s])):

                for prob, next_state, reward, done in mdp[s][a]:
                    Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))

        delta = np.max(np.abs(V - np.max(Q, axis=1)))

        if delta < epsilon:
            break

        V = np.max(Q, axis=1)
    
    # Extract the optimal policy by selecting the action that maximizes Q-value for each state
    pi_temp = {s: np.argmax(Q[s]) for s in range(len(mdp))}
    
    return pi_temp, V, count
    

In [151]:
#Function to print the policy you got after running the policy iteration or value iteration on the 4x4 FrozenLake environment
def print_policy(policy, env):
    """
    Prints the policy for the 4x4 FrozenLake environment in a grid layout.
    """
    action_symbols = {0: '←', 1: '↓', 2: '→', 3: '↑'}  #action symbols
    grid_size = env.desc.shape  #get the grid dimensions (e.g., 4x4)
    
    policy_symbols = np.array([action_symbols[action] for cell,action in policy.items()])
    policy_grid = policy_symbols.reshape(grid_size)  #reshape into a grid

    print("Policy Grid:")
    for row in policy_grid:
        print(" ".join(row))


In [ ]:
pi1, val1, count1 = policy_iteration(mdp_transitions)
pi2, val2, count2 = value_iteration(mdp_transitions)
print_policy(pi1,env)
print_policy(pi2,env)
# env.reset()
# env.step(0)


Policy Grid:
← ↑ ↑ ↑
← ← ← ←
↑ ↓ ← ←
← → ↓ ←
Policy Grid:
← ↑ ↑ ↑
← ← ← ←
↑ ↓ ← ←
← → ↓ ←


(4, 0.0, False, False, {'prob': 0.3333333333333333})

You can also write a function `test_policy()` to test your policy after training to find the number of times you reached the goal state

In [164]:
# def test_policy(pi, env, goalstate):
#     # Complete this function to test the policy
#     return
def test_policy(pi, env, goalstate):
    """
    Test a given policy in the environment until the goalstate is reached.

    Args:
        pi (dict): The optimal policy mapping each state to an action.
        env (object): The environment object that provides the `step` method.
        goalstate (int): The goal state that the agent is trying to reach.

    Returns:
        steps (int): The number of steps taken to reach the goal state.
    """
    state = 0 # Reset the environment to get the initial state
    steps = 0
    
    while state != goalstate:
        action = pi[state]  # Get the action for the current state from the policy
        next_state, reward, done, _, __ = env.step(action)  # Take the action and get the result
        
        # If the next state is a dictionary, extract the state identifier
        if isinstance(next_state, dict):
            next_state = next_state.get('state')  # Assuming 'state' is the key for the state ID
        
        state = next_state  # Move to the next state
        steps += 1
        
        if done:
            break  # If the environment signals done, stop

    return steps

test_policy(pi1, env, 15)


32